<a href="https://colab.research.google.com/github/ykim71/georgia_analysis/blob/main/(2)_Georgia_place_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# need access to the folder(s) below:

#%cd /content/drive/MyDrive/Georgia/
#%cd /content/drive/MyDrive/Georgia/Locations Analysis


# Location analysis

> Find place and location information in Georgia through NER


> Locate exact address assisted by Google Map API through R library 





## Prep for place analysis

In [ ]:
%cd /content/drive/MyDrive/Georgia/Locations Analysis


In [ ]:
import pandas as pd

df = pd.read_pickle('Georgia_data_v2_translation_ner3_lem.pkl') #CO: I assume this is the relevant file!

In [ ]:
# lemmatization if needed
import spacy
from spacy import displacy

sp = spacy.load('en_core_web_sm', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])


In [ ]:
def text_ner_place(text):
  text = sp(text)
  
  ner_list = []
  
  for entity in text.ents:
    if entity.label_ == "FAC":
      ner_list.append(entity.text + " (" + entity.label_ + ")")
    if entity.label_ == "GPE":
      ner_list.append(entity.text + " (" + entity.label_ + ")")
    if entity.label_ == "LOC":
      ner_list.append(entity.text + " (" + entity.label_ + ")")
    else:
      None
  return str(set(ner_list))


In [ ]:
df['ner_places'] = df['article_final'].astype(str).apply(text_ner_place)

In [ ]:
print(df)

In [ ]:
#df.to_pickle('Georgia_data_v2_translation_ner3_lem.pkl') 

## Find rough location from spaCy

In [ ]:
import pandas as pd

df = pd.read_pickle('Georgia_data_v2_translation_ner3_lem.pkl')


In [ ]:
df_ner = df.loc[df['ner_places'] != 'set()'] 

In [ ]:
print(len(df))
print(len(df_ner))

In [ ]:
df_expand = df_ner['ner_places'].str.split(',').explode().str.strip('{} ')


In [ ]:
df_expand = pd.DataFrame(df_expand)
print(df_expand) # note that there's error of spaCy that capture places and person in GPE/LOC/FAC and PERSON 

In [ ]:
df_expand['ner_places_clean'] = df_expand['ner_places'].str.replace(r"\'","")
df_expand['ner_places_clean'] = df_expand['ner_places_clean'].str.replace(r" \"","")
df_expand['ner_places_clean'] = df_expand['ner_places_clean'].str.replace(r"\"","")
df_expand['ner_places_clean'] = df_expand['ner_places_clean'].str.replace(r"\(.*\)","")

In [ ]:
df_expand_count = df_expand.groupby('ner_places_clean').count().reset_index()
df_expand_count 

In [ ]:
# NER entities appeared >= 5 across articles 

df_freq_n5 = df_expand_count[df_expand_count["ner_places"] >= 5]
df_freq_n5.to_excel('df_freq_n5_ner_v2.xlsx')


In [ ]:
# # compare place entities with v1

# df_freq_n5_v2 = pd.read_excel('df_freq_n5_ner_v2.xlsx')
# df_freq_n5_v1 = pd.read_excel('df_freq_n5_ner.xlsx')

# df_freq_n5_v1['version'] = 'v1'
# df_freq_n5_v2['version'] = 'v2'

# df_freq_n5_merge = pd.merge( df_freq_n5_v1,df_freq_n5_v2, how="outer", on=["ner_places_clean"])

## Find sentence example of location

In [ ]:
import pandas as pd #CO: already imported.

#ner_places = pd.read_csv('Georgia_ner_places.csv') # previous data
ner_places = pd.read_csv('Locations Analysis/Georgia_ner_places_v2.xlsx') # updated data; added all possible address from Google Map API

In [ ]:
ner_places['ner_places_clean'] = ner_places['ner_places_clean'].str.rstrip()

In [ ]:
ner_places_all = list(set(ner_places['ner_places_clean'].tolist())) 

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
# df["id"] = df.index + 1
df_ner = df.loc[df['ner_places'] != 'set()'] #CO: This step has already been done once above (in chunk 21)

In [ ]:

import numpy as np

df_ner['place_ng1'] = df_ner['article_final'].astype(str).apply(lambda x: [i for i in ner_places_all if i in x]) # make a separate column to search those words in another article column


In [ ]:
text_df_uni_dict = df_ner[['id',  'place_ng1']]
text_df_uni_article = df_ner[['id', "article_final"]]


In [ ]:
#CO: not sure why we're taking a sample sentence here. Can't tell based on this code how this is a sample.
sample_sentence = text_df_uni_dict['place_ng1'].astype(str).str.split(',').explode().str.strip('[] ')
sample_sentence_df = pd.DataFrame(sample_sentence)

In [ ]:

merge_uni = pd.merge(sample_sentence_df, text_df_uni_article, left_index=True, right_index=True)

merge_uni['place_ng1'] = merge_uni['place_ng1'].str.strip('\'')
#CO: recommend printing merge_uni to check that it looks as you expect. Same for the objects created in the next few chunks.

In [ ]:
merge_uni_count = merge_uni.groupby('place_ng1').count() #


In [ ]:
# articles mentioned a location less than 3
merge_uni_count_l3 = merge_uni_count[merge_uni_count['article_final'] < 3].reset_index() # the threshold is for sample arbitrary 3 sentences for manual checking;


In [ ]:
merge_uni_count_l3_places = merge_uni_count_l3.place_ng1.values.tolist()
merge_uni_count_l3_places

In [ ]:
merge_uni_l3 = merge_uni[merge_uni['place_ng1'].isin(merge_uni_count_l3_places)]
merge_uni_l3_group = merge_uni_l3.groupby('place_ng1').apply(lambda x: x.sample(3, replace=True)).reset_index(drop=True)

In [ ]:
# articles mentioned a location more than 3 --> random sample

merge_uni_m3 = merge_uni[~merge_uni['place_ng1'].isin(merge_uni_count_l3_places)]
merge_uni_m3_group = merge_uni_m3.groupby('place_ng1').apply(lambda x: x.sample(3, replace=False)).reset_index(drop=True)


In [ ]:
# merge 3 samples + those with less than 3 
merge_uni_m3_group2 = pd.concat([merge_uni_m3_group, merge_uni_l3_group]).reset_index()

In [ ]:
merge_uni_m3_group2.rename(columns={'place_ng1':'places'}, inplace=True)


In [ ]:
merge_uni_m3_group2

In [ ]:
# split an article into sentences

from nltk.tokenize import sent_tokenize
by_sentence = merge_uni_m3_group2['article_final'].apply(lambda x : sent_tokenize(x)).apply(pd.Series,1).stack()
by_sentence_df = pd.DataFrame(by_sentence)
by_sentence_df.rename(columns={0:'sentence_each'}, inplace=True)

by_sentence_df['sentence_turn'] = by_sentence_df.index.get_level_values(1)
by_sentence_df.index = by_sentence_df.index.get_level_values(0)


In [ ]:
by_sentence_df

In [ ]:
# merging split sentences to the article level
merge_all_m3_join = pd.concat([merge_uni_m3_group2, by_sentence_df], axis=1)


In [ ]:
merge_all_m3_join['search_words'] = merge_all_m3_join.apply(lambda x: x.places in x.sentence_each, axis=1)


In [ ]:
merge_all_m3_join['article_id'] = merge_all_m3_join.index.get_level_values(0)
merge_all_m3_join.reset_index(drop=True, inplace=True)


In [ ]:
# find those sentences that contain searching words (locations)

merge_all_m3_join_TRUE = merge_all_m3_join[merge_all_m3_join['search_words']==True]

In [ ]:
merge_temp = merge_all_m3_join_TRUE.groupby('places')['id'].count() # first empty row is because of white space considered as a word


In [ ]:
# extract 3 random sentences containing the searching word (location) only

merge_temp_m3_group = merge_all_m3_join_TRUE.groupby('places').apply(lambda x: x.sample(3, replace=False)).reset_index(drop=True)


In [ ]:
merge_temp_m3_group

In [ ]:
merge_temp_m3_group2 = merge_temp_m3_group[['places','id','sentence_each']]

In [ ]:
merge_temp_m3_group2['domainId'] = merge_temp_m3_group2.groupby('places')['places'].rank(method='first')


In [ ]:
merge_temp_m3_group2.sort_values('places')

In [ ]:
# spreading dataset with 3 different sentences as columns

merge_temp_m3_group2_pivot = merge_temp_m3_group2.pivot(index='places', columns = 'domainId', values='sentence_each').reset_index() 

In [ ]:
merge_temp_m3_group2_pivot.rename(columns={1:'sent_1', 2:'sent_2', 3:'sent_3'}, inplace=True)
merge_temp_m3_group2_pivot 

In [ ]:
ner_places['places'] = ner_places['ner_places_clean']

In [ ]:
merge_temp_final = pd.merge(ner_places, merge_temp_m3_group2_pivot, on="places", how='left').reset_index()
#CO: recommend printing this. Does it look as expected?

In [ ]:
merge_temp_final['sent_1'] = merge_temp_final['sent_1'].str.replace('\n', ' ')
merge_temp_final['sent_2'] = merge_temp_final['sent_2'].str.replace('\n', ' ')
merge_temp_final['sent_3'] = merge_temp_final['sent_3'].str.replace('\n', ' ')

In [ ]:
merge_temp_final.to_excel('/content/drive/MyDrive/Georgia/Locations Analysis/Georgia_ner_places_v2_wsent.xlsx') 

## Look up location (w updated data) on articles

In [ ]:
%cd /content/drive/MyDrive/Georgia/Locations Analysis

In [ ]:
# data created by manual coding
import pandas as pd

location_final = pd.read_excel('Georgia_place_validation_counties.xlsx') # these are only places in GA

In [ ]:
location_final = location_final[location_final['final_selection']==1]

In [ ]:
location_final_st = location_final[['ner_place_clean','county_level', 'final_selection']]

In [ ]:
location_final_st = location_final_st.dropna()

In [ ]:
print(len(location_final)) #CO: include this, so we can tell how many are being dropped
print(len(location_final_st))

In [ ]:
location_final_st

In [ ]:
# # multiple counties deleted
# location_final_st = location_final_st[location_final_st['county_level'].str.contains('Counties')==False]

In [ ]:
len(set(location_final_st.county_level.values.tolist()))

In [ ]:
location_final_list = list(set(location_final_st['ner_place_clean'].tolist()))

In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/Georgia/Web Scraping/Georgia_data_v2_translation_ner2_lem.pkl')


In [ ]:

import re

def bigrams(s):
    return [i for i in s if re.search(r'\s', i) ]
  
def unigrams(s):
    return [i for i in s if not re.search(r'\s', i) ]

word_bigrams_done = list(set(bigrams(location_final_list)))
print(word_bigrams_done) #CO: some of these don't really seem to be bigrams. For example: 'Mill Creek High School Football Game'
print(len(word_bigrams_done))
                       
word_unigrams_done = unigrams(location_final_list)
print(word_unigrams_done)
print(len(word_unigrams_done)) #CO: why are there more bigrams than unigrams? Conceptually, wouldn't we expect at least as mnay unigrams? I'm probably just missing something, but I'll mention it in case it's helpful to take a second look at!


In [ ]:
import numpy as np 

df['place_ng1'] = df['article_final'].astype(str).apply(lambda x: set.intersection(set(x.split(' ')), word_unigrams_done))
df['place_ng2'] = df['article_final'].astype(str).apply(lambda x: [i for i in word_bigrams_done if i in x])


In [ ]:
df_uni = df[(df['place_ng1'].str.len()) >= 1]
df_bi = df[(df['place_ng2'].str.len()) >= 1]

df_uni = df_uni[['id', 'source', 'place_ng1']]
df_bi = df_bi[['id', 'source', 'place_ng2']]

df_uni_df = df_uni.explode('place_ng1')
df_bi_df = df_bi.explode('place_ng2')

df_uni_df = df_uni_df.rename(columns={'place_ng1': 'place_ner'})
df_bi_df = df_bi_df.rename(columns={'place_ng2': 'place_ner'})


In [ ]:
df_uni_df #CO: most these seem to be names of people (Allen, Charles, Rebecca, Philips, etc.), not places. 
df_bi_df #CO: these generally look more relevant than the unigrams

In [ ]:
df_ng_all = pd.concat([df_uni_df, df_bi_df])


In [ ]:
count = df_ng_all.groupby(['source'])['place_ner'].count() 


In [ ]:
count_20 = count[count['place_ner'] >= 20].reset_index() # 20 as an arbitrary reference to cut
count_20


In [ ]:
count_20_id = list(set(count_20['source'].tolist()))

df_ng_all_m20 = df_ng_all[df_ng_all['source'].isin(count_20_id)]
df_ng_all_m20 


In [ ]:
len(count_20_id)

In [ ]:
location_final_st = location_final_st.rename(columns={'ner_place_clean': 'place_ner'})
location_final_st #CO: should the two columns correspond to each other? If so, how do we know that they do? Is this what was manually validated earlier on?


In [ ]:
df_ng_all_m20_county = pd.merge(df_ng_all_m20, location_final_st, how='left', on='place_ner')
df_ng_all_m20_county

In [ ]:
df_ng_all_m20_county

In [ ]:
# drop duplicates to count the same county with different places 

df_ng_all_m20_county_unique = df_ng_all_m20_county.drop_duplicates(subset=['id', 'source', 'county_level']) 

In [ ]:
df_ng_all_m20_county_unique

In [ ]:
df_ng_all_m20_county_unique_count = df_ng_all_m20_county_unique.groupby(['source','county_level'])['id'].count().reset_index()
df_ng_all_m20_county_unique_count

In [ ]:
# rename article count back

df_ng_all_m20_county_unique_count = df_ng_all_m20_county_unique_count.rename(columns={'id': 'article_id_count'})
df_ng_all_m20_county_unique_count


In [ ]:
#df_ng_all_m20_county_unique_count.to_excel('Georgia_m20_county_unique_count_v2.xlsx')
